<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Statsmodels" data-toc-modified-id="Statsmodels-1">Statsmodels</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Just-poll_changed" data-toc-modified-id="Just-poll_changed-1.0.1">Just poll_changed</a></span></li><li><span><a href="#With-poll_changed-and-farther/closer" data-toc-modified-id="With-poll_changed-and-farther/closer-1.0.2">With poll_changed and farther/closer</a></span></li></ul></li><li><span><a href="#Getting-donut-df" data-toc-modified-id="Getting-donut-df-1.1">Getting donut df</a></span></li><li><span><a href="#Checking-2012-data" data-toc-modified-id="Checking-2012-data-1.2">Checking 2012 data</a></span></li></ul></li></ul></div>

# Statsmodels

In [2]:
%matplotlib inline
import pandas as pd 
import numpy as np
import seaborn as sns
import statsmodels as sm
import statsmodels.formula.api as smf
import statsmodels.regression.linear_model as smlm

In [3]:
df = pd.read_csv('/Users/jakemainwaring/Desktop/paramtesting_data/hail_mary.zip', sep='\t')

In [4]:
df['is_closer'] = np.where(df['delta_dist'] < 0, 1, 0) 
df['is_farther'] = np.where(df['delta_dist'] > 0, 1, 0) 

### Just poll_changed

In [5]:
mod = smf.ols('voted ~ poll_changed', data=df)
res = mod.fit(cov_type='cluster',  cov_kwds={'groups': df['block']})
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  voted   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     7.276
Date:                Wed, 15 May 2019   Prob (F-statistic):            0.00700
Time:                        12:54:51   Log-Likelihood:                -98399.
No. Observations:              224022   AIC:                         1.968e+05
Df Residuals:                  224020   BIC:                         1.968e+05
Df Model:                           1                                         
Covariance Type:              cluster                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.8307      0.002    434.846   

### With poll_changed and farther/closer

In [6]:
mod = smf.ols('voted ~ is_closer + is_farther', data=df)
res = mod.fit(cov_type='cluster',  cov_kwds={'groups': df['block']})
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  voted   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     16.93
Date:                Wed, 15 May 2019   Prob (F-statistic):           4.58e-08
Time:                        12:55:00   Log-Likelihood:                -98384.
No. Observations:              224022   AIC:                         1.968e+05
Df Residuals:                  224019   BIC:                         1.968e+05
Df Model:                           2                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8307      0.002    434.845      0.0

## Getting donut df

In [17]:
# Filtering down
unique_blocks = df[np.abs(df['delta_dist']) > 1]['block'].unique()
donut_df = df[df['block'].isin(unique_blocks)]

In [18]:
donut_df['is_closer'] = np.where(donut_df['delta_dist'] < 0, 1, 0) 
donut_df['is_farther'] = np.where(donut_df['delta_dist'] > 0, 1, 0) 

/Users/jakemainwaring/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/jakemainwaring/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [21]:
donut_df.to_csv('hailmary_donut.csv')

## Checking 2012 data

In [7]:
twenty_twelve = pd.read_csv('/Users/jakemainwaring/Downloads/2012_votes.csv')

In [8]:
df.head(2)

,ncid,poll_changed,voted,delta_dist,voting_method,block,is_closer,is_farther
0,DP16209,0.0,1.0,0.0,ABSENTEE ONESTOP,0,0,0
1,DP13039,0.0,1.0,0.0,IN-PERSON,0,0,0


In [9]:
twenty_twelve.head(2)

,ncid,voted_2012
0,BY25079,1.0
1,BY250876,1.0


In [10]:
twenty_twelve_merged = pd.merge(twenty_twelve, df, on='ncid', how='inner')

In [11]:
twenty_twelve_merged.head(2)

,ncid,voted_2012,poll_changed,voted,delta_dist,voting_method,block,is_closer,is_farther
0,BY251461,1.0,0.0,1.0,0.0,ABSENTEE BY MAIL,6086,0,0
1,BY251461,1.0,0.0,1.0,0.0,ABSENTEE BY MAIL,6087,0,0


In [14]:
twenty_twelve_merged.to_csv('hailmary_point_three.csv')

In [12]:
mod = smf.ols('voted_2012 ~ poll_changed', data=twenty_twelve_merged)
res = mod.fit()#  cov_kwds={'groups': twenty_twelve_merged['block']})
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             voted_2012   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     203.8
Date:                Wed, 15 May 2019   Prob (F-statistic):           3.30e-46
Time:                        12:55:51   Log-Likelihood:                -48065.
No. Observations:              224016   AIC:                         9.613e+04
Df Residuals:                  224014   BIC:                         9.615e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.9020      0.001   1390.445   

In [13]:
mod = smf.ols('voted_2012 ~ is_farther + is_closer', data=twenty_twelve_merged)
res = mod.fit()#  cov_kwds={'groups': twenty_twelve_merged['block']})
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             voted_2012   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     122.0
Date:                Wed, 15 May 2019   Prob (F-statistic):           1.07e-53
Time:                        12:57:05   Log-Likelihood:                -48044.
No. Observations:              224016   AIC:                         9.609e+04
Df Residuals:                  224013   BIC:                         9.613e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9020      0.001   1390.567      0.0

In [11]:
mod = smf.ols('voted_2012 ~ poll_changed + C(block)', data=twenty_twelve_merged)
res = mod.fit()#  cov_kwds={'groups': twenty_twelve_merged['block']})
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             voted_2012   R-squared:                       0.168
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     1.545
Date:                Tue, 14 May 2019   Prob (F-statistic):          1.25e-133
Time:                        16:15:53   Log-Likelihood:                -7771.3
No. Observations:               55714   AIC:                         2.846e+04
Df Residuals:                   49255   BIC:                         8.613e+04
Df Model:                        6458                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.7824      0.099  

In [18]:
mod = smf.ols('voted_2012 ~ poll_changed', data=twenty_twelve_merged)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             voted_2012   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     139.0
Date:                Tue, 14 May 2019   Prob (F-statistic):           4.78e-32
Time:                        15:33:17   Log-Likelihood:                -12841.
No. Observations:               55714   AIC:                         2.569e+04
Df Residuals:                   55712   BIC:                         2.570e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.9017      0.001    656.769   

In [12]:
mod = smf.ols('voted_2012 ~ is_closer + is_farther', data=twenty_twelve_merged)
res = mod.fit(cov_type='cluster',  cov_kwds={'groups': twenty_twelve_merged['block']})
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             voted_2012   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     54.97
Date:                Tue, 14 May 2019   Prob (F-statistic):           2.12e-24
Time:                        15:19:04   Log-Likelihood:                -12834.
No. Observations:               55714   AIC:                         2.567e+04
Df Residuals:                   55711   BIC:                         2.570e+04
Df Model:                           2                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9017      0.002    505.878      0.0

In [14]:
twenty_twelve_merged.head(2)

,ncid,voted_2012,Unnamed: 0,poll_changed,voted,delta_dist,block,is_closer,is_farther
0,BY173096,1.0,1554858,1.0,1.0,0.922897,1124,0,1
1,BY40892,1.0,1554967,1.0,1.0,-0.016144,5507,1,0


In [28]:
blocked_df = twenty_twelve_merged[twenty_twelve_merged['poll_changed']==0]
treatment_df = twenty_twelve_merged[twenty_twelve_merged['poll_changed']==1][['block', 'voted_2012']]

blocked_mean = pd.DataFrame(blocked_df.groupby('block')['voted_2012'].mean()).reset_index()
full_mean = pd.merge(blocked_df, treatment_df, how='inner', on='block')
full_mean['vote_diff'] = full_mean['voted_2012_y'] - full_mean['voted_2012_x']
full_mean['vote_diff'].mean()

-0.0575454840805718

In [29]:
blocked_df = twenty_twelve_merged[twenty_twelve_merged['is_farther']==0]
treatment_df = twenty_twelve_merged[twenty_twelve_merged['is_farther']==1][['block', 'voted_2012']]

blocked_mean = pd.DataFrame(blocked_df.groupby('block')['voted_2012'].mean()).reset_index()
full_mean = pd.merge(blocked_df, treatment_df, how='inner', on='block')
full_mean['vote_diff'] = full_mean['voted_2012_y'] - full_mean['voted_2012_x']
full_mean['vote_diff'].mean()

-0.06997523278051228

In [30]:
blocked_df = twenty_twelve_merged[twenty_twelve_merged['is_closer']==0]
treatment_df = twenty_twelve_merged[twenty_twelve_merged['is_closer']==1][['block', 'voted_2012']]

blocked_mean = pd.DataFrame(blocked_df.groupby('block')['voted_2012'].mean()).reset_index()
full_mean = pd.merge(blocked_df, treatment_df, how='inner', on='block')
full_mean['vote_diff'] = full_mean['voted_2012_y'] - full_mean['voted_2012_x']
full_mean['vote_diff'].mean()

-0.03469540660480664